# Analysis of Turkish Government's Bonds

This notebook is written to quickly analyse YTM (yield to maturity), and CY (current yield) of Turkish Government's long term bonds. Prices are fetched from Luxemburg Stock Exchange. Prices may differ in your local broker.

When calculating YTM it is assumed that:
* bonds are hold until maturity date,
* every coupon is reinvested in the same bond.

In [ ]:
!pip install bond-pricing

Next thing is to fetch bond data along with the latest prices from [Luxembourg Stock Exchange](https://www.bourse.lu) and analyse the bonds.

In [ ]:
from bond_pricing.simple_bonds import bond_price_breakup, bond_yield
from datetime import date, datetime
import requests
from prettytable import PrettyTable

r = requests.get("https://www.bourse.lu/api/issuer/securities/26760")
r.raise_for_status()
body = r.json()
table = PrettyTable(
    [
        "ISIN",
        "Currency",
        "Rate",
        "Maturity",
        "Price",
        "Accrued Interest",
        "Dirty Price",
        "Next Coupon",
        "YTM",
        "Current Yield",
    ]
)
today = date.today()
print(f"Updated on {today}")
for s in body["list"]:
    freq = 2 if s["currency"] == "USD" else 1
    rate = float(s["name"].split()[1][0:-1].replace(",", ".")) / 100.0
    coupon = 100.0 * rate
    maturity = date.fromtimestamp(s["finalMaturityDate"] / 1000 + 3600)
    price = s["pricesSummary"]["latestPrice"]["amount"]
    prices = bond_price_breakup(settle=today, cpn=rate, mat=maturity, freq=freq)
    ytm = bond_yield(
        settle=today, cpn=rate, mat=maturity, price=price, freq=freq, guess=0.05
    )
    table.add_row(
        [
            s["isinCode"],
            s["currency"],
            round(rate * 100, 3),
            maturity,
            price,
            round(prices["AccruedInterest"], 3),
            round(price + prices["AccruedInterest"], 3),
            prices["NextCoupon"].date(),
            round(ytm * 100, 3),
            round(coupon * 100 / price, 3),
        ]
    )

print(table.get_string(sortby="YTM"))


/usr/local/lib/python3.7/dist-packages/bond_pricing/simple_bonds.py:528: UserWarning: Module isda_daycounters is not installed.
Only 'simple' daycount (basically ACT/365) is available.
To use other daycounts, install isda_daycounters from
https://github.com/miradulo/isda_daycounters
  warn("Module isda_daycounters is not installed.\n"


Updated on 2021-03-27
+--------------+----------+--------+------------+---------+------------------+-------------+-------------+--------+---------------+
|     ISIN     | Currency |  Rate  |  Maturity  |  Price  | Accrued Interest | Dirty Price | Next Coupon |  YTM   | Current Yield |
+--------------+----------+--------+------------+---------+------------------+-------------+-------------+--------+---------------+
| US900123BH29 |   USD    | 5.625  | 2021-03-29 | 100.082 |      2.759       |   102.841   |  2021-03-29 | -2.826 |      5.62     |
| US900123CN87 |   EUR    |  3.25  | 2025-06-13 | 100.553 |      2.555       |   103.108   |  2021-06-13 | 3.106  |     3.232     |
| XS0993155398 |   EUR    |  4.35  | 2021-11-12 | 100.692 |      1.609       |   102.301   |  2021-11-12 | 3.197  |      4.32     |
| XS1057340009 |   EUR    | 4.125  | 2023-04-10 | 100.195 |      3.967       |   104.162   |  2021-04-10 | 4.022  |     4.117     |
| US900123BY51 |   USD    | 5.125  | 2022-03-25 | 100.